<a href="https://colab.research.google.com/github/navneet2263/ai-ml-project/blob/main/HousePricePredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_california_housing

#Independent data


In [ ]:
data=fetch_california_housing()
print(data.DESCR)
df=pd.DataFrame(data = data.data, columns= data.feature_names)
df.head()


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [ ]:
#Dependent Data
df['Target'] = data.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
# EDA
!pip install numpy==1.26.4
!pip install sweetviz


In [ ]:
import sweetviz as sv  #does EDA
report = sv.analyze(df)
report.show_html("./report.html")

                                             |          | [  0%]   00:00 -> (? left)

Report ./report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
#DATA PREPROCESSING
# Feature Engineering
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")  #api will give the location of the city for the given coordinates#


In [ ]:
from geopy.geocoders import Nominatim
# Re-initialize geolocator with a more specific user agent to avoid potential blocks
geolocator = Nominatim(user_agent="colab_california_housing_analyzer")
geolocator.reverse("37.88" + " , " + "-122.23")

Location(Grizzly Peak Boulevard, Oakland, Alameda County, California, 94720, United States, (37.8803854, -122.2301805, 0.0))

In [ ]:
from types import NoneType
def location(cord):
  Latitude = str(cord[0])
  Longitude = str(cord[1])

  location = geolocator.reverse(Latitude+","+Longitude).raw['address']  #raw returns a dictionary

  #if values are missing replace by a empty string

  if location.get('Road') is None:
    location['Road'] = None

  if location.get('County') is None:
    location['County'] = None

  loc_update['County'].append(location['County'])
  loc_update['Road'].append(location['Road'])


In [ ]:


'''loc_update = {"County":[],
              "Road":[],
              "Neighbourhood":[]}

for i,cord in enumerate(df.iloc[:,6:-1].values):

  location(cord)

  pickle.dump(loc_update, open('loc_update.pickle','wb'))

  if i%100==0:
    print(i)'''


'loc_update = {"County":[],\n              "Road":[],\n              "Neighbourhood":[]}\n\nfor i,cord in enumerate(df.iloc[:,6:-1].values):\n\n  location(cord)\n\n  pickle.dump(loc_update, open(\'loc_update.pickle\',\'wb\'))\n\n  if i%100==0:\n    print(i)'

In [ ]:
import pickle

loc_update = pickle.load(open("/content/loc_update (1).pickle","rb"))
loc_update['county']


In [ ]:
loc = pd.DataFrame(loc_update)
loc.head()

,county,road
0,Alameda County,Panoramic Way
1,Alameda County,Caldecott Lane
2,Alameda County,Grove Shafter Freeway
3,Alameda County,Florio Street
4,Alameda County,Florio Street


In [ ]:
loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   county  15262 non-null  object
 1   road    19498 non-null  object
dtypes: object(2)
memory usage: 322.6+ KB


In [ ]:
# add the new features to our dataframe

for i in loc_update.keys():
  df[i] = loc_update[i]

df = df.sample(axis = 0, frac = 1)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target,county,road
5917,1.9688,37.0,4.017021,1.038298,1107.0,4.710638,34.28,-118.44,1.441,None,Chatsworth Drive
3354,3.5962,15.0,5.923503,1.110865,2503.0,2.774945,40.50,-120.67,0.859,Lassen County,None
11897,7.2519,33.0,7.526515,1.037879,1398.0,2.647727,33.96,-117.37,2.166,Riverside County,Victoria Hill Drive
18481,5.5936,21.0,5.946369,1.014525,3011.0,3.364246,37.06,-121.61,3.868,Santa Clara County,Santa Teresa Boulevard
14119,1.6034,30.0,3.851927,1.107505,1620.0,3.286004,32.75,-117.09,0.844,San Diego County,Estrella Avenue


In [ ]:
# drop lat,long
df=df.drop(labels=["Latitude","Longitude"],axis=1)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Target,county,road
5917,1.9688,37.0,4.017021,1.038298,1107.0,4.710638,1.441,None,Chatsworth Drive
3354,3.5962,15.0,5.923503,1.110865,2503.0,2.774945,0.859,Lassen County,None
11897,7.2519,33.0,7.526515,1.037879,1398.0,2.647727,2.166,Riverside County,Victoria Hill Drive
18481,5.5936,21.0,5.946369,1.014525,3011.0,3.364246,3.868,Santa Clara County,Santa Teresa Boulevard
14119,1.6034,30.0,3.851927,1.107505,1620.0,3.286004,0.844,San Diego County,Estrella Avenue


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20640 entries, 5917 to 15482
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Target      20640 non-null  float64
 7   county      15262 non-null  object 
 8   road        19498 non-null  object 
dtypes: float64(7), object(2)
memory usage: 1.6+ MB


In [ ]:
#using classification algorithm to fil the missing categorical values
#applying logistic regression to find the missing value

missing_idx=[]

for i in range(df.shape[0]):

  if df['road'][i] is None:
    missing_idx.append(i)

# Independent Parameters
missing_road_X_train = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0])  if i not in missing_idx])

# Dependent Parameters
missing_road_y_train = np.array([df['road'][i] for i in range(df.shape[0]) if i not in missing_idx])

missing_road_X_test = np.array([[df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0]) if i in missing_idx])



In [ ]:
from sklearn.linear_model import SGDClassifier

# model initialization

model_1= SGDClassifier()

# model Training

model_1.fit(missing_road_X_train,missing_road_y_train)

missing_road_y_pred = model_1.predict(missing_road_X_test)


/tmp/ipython-input-2344263149.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['road'][missing_idx[i]] = missing_road_y_pred[i]
/tmp/ipython-input-2344263149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli